<a href="https://colab.research.google.com/github/TakumiHS/keio-quant-macro-takumikumada/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rtfpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['k_pc'] = np.log(data['rtfpna'] / data['emp'])  # TFP Growth
data['labsh'] #TFP share
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita

# Calculate Capital deepening
data['cd'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of capital per capita

# Calculate TFP Growth
data['tg'] = data['g'] - data['cd'] #TFP Growth

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean', 'tg': "mean", 'cd': 'mean', 'labsh': 'mean', 'a': 'mean'})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['TFP Growth'] = summary['tg']
summary['Capital Deepening'] = summary['cd']
summary['TFP Share'] = summary['labsh']
summary['Capital Share'] = summary['a']

# Print output
print(summary)



                    g        tg        cd     labsh         a  Growth Rate  \
countrycode                                                                  
CAN          0.887700  2.224496 -1.336796  0.653810  0.346190     0.887700   
DEU          0.661055  1.008208 -0.347152  0.629270  0.370730     0.661055   
FRA          0.829236  1.515931 -0.686695  0.618806  0.381194     0.829236   
GBR          1.015544  1.829163 -0.813619  0.587007  0.412993     1.015544   
ITA         -0.037181  1.360500 -1.397681  0.515165  0.484835    -0.037181   
JPN          0.710956  0.708473  0.002484  0.569399  0.430601     0.710956   
USA          1.536985  1.764296 -0.227310  0.606740  0.393260     1.536985   

             TFP Growth  Capital Deepening  TFP Share  Capital Share  
countrycode                                                           
CAN            2.224496          -1.336796   0.653810       0.346190  
DEU            1.008208          -0.347152   0.629270       0.370730  
FRA          